## Extract data

In [75]:
# required packages
import fxcmpy
import socketio

import pandas as pd
import numpy as np
import os

import tqdm

In [2]:
# create connection
con = fxcmpy.fxcmpy(config_file='fxcm_pp.cfg')

In [78]:
# list of instruments need to extract fx-data for

#instruments = con.get_instruments()
curr_pairs = ['EUR/USD',
             'GBP/USD',
             'USD/CHF',
             'AUD/USD',
             'USD/CAD',
             'NZD/USD',
             'EUR/CHF',
             'EUR/GBP',
             'EUR/AUD',
             'EUR/CAD']

In [84]:
# create folders to save data - only needs to run once to create folders
for curr_pair in curr_pairs:

    newpath = f'./data/fx_data/{curr_pair[:3]}_{curr_pair[4:]}'

    if not os.path.exists(newpath):
         os.makedirs(newpath)

In [85]:
# get data and store
for curr_pair in tqdm.tqdm(curr_pairs):

    #print(f'extracting data for {curr_pair}\n')
    # folder path to store the data
    folder_path = f'./data/fx_data/{curr_pair[:3]}_{curr_pair[4:]}/'
    
    # extract new data
    new_fx_data = con.get_candles(curr_pair
                              , period='m15'
                              , number = 10000 # get last 10,000 records
                              , with_index=False)
    
    # load exsisting data
    try:
        prvs_fx_data = pd.read_parquet(folder_path+f'fx_data_{curr_pair[:3]}_{curr_pair[4:]}.parquet')
    except:
        prvs_fx_data = new_fx_data
    
    # concat old and new data + drop duplicates
    data_export = pd.concat([prvs_fx_data,new_fx_data])
    
    dup_flag = True  
    while dup_flag:
        data_export.drop_duplicates(subset=['date'],inplace=True)
        data_export.reset_index(inplace=True, drop=True)
        
     
        if data_export.date.duplicated().sum() == 0:
            dup_flag = False

    # sort by date
    data_export['date'] = pd.to_datetime(data_export['date'])
    data_export.sort_values('date', inplace = True)
    
    # store new data
    data_export.to_parquet(folder_path+f'fx_data_{curr_pair[:3]}_{curr_pair[4:]}.parquet', index = False)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.46it/s]
